In [4]:

# API経由で為替レートのストリーミングを取得する
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
%matplotlib qt

In [5]:
# API接続設定のファイルを読み込む
import configparser

# 設定
config = configparser.ConfigParser()
config.read('./config_v1.txt') # パスの指定が必要です
account_ID = config['oanda']['account_id']
api_key = config['oanda']['api_key']

In [6]:
#自分のアカウント、トークンをセット

api = API(access_token=api_key, environment="practice")

In [7]:
#FX API経由で現時点の為替レートのストリーミングを取得しましょう。
#まずはAPIへ投げる設定を count=200 、 granularity=M5 と作ってあげます
params = {
  "count": 4000,
  "granularity": "M1"
}

In [8]:

# APIから為替レートのストリーミングを取得
r = instruments.InstrumentsCandles(instrument="USD_JPY", params=params)
api.request(r)

{'instrument': 'USD_JPY',
 'granularity': 'M1',
 'candles': [{'complete': True,
   'volume': 86,
   'time': '2021-01-06T02:13:00.000000000Z',
   'mid': {'o': '102.782', 'h': '102.794', 'l': '102.780', 'c': '102.790'}},
  {'complete': True,
   'volume': 80,
   'time': '2021-01-06T02:14:00.000000000Z',
   'mid': {'o': '102.790', 'h': '102.792', 'l': '102.780', 'c': '102.788'}},
  {'complete': True,
   'volume': 96,
   'time': '2021-01-06T02:15:00.000000000Z',
   'mid': {'o': '102.786', 'h': '102.800', 'l': '102.782', 'c': '102.785'}},
  {'complete': True,
   'volume': 70,
   'time': '2021-01-06T02:16:00.000000000Z',
   'mid': {'o': '102.786', 'h': '102.788', 'l': '102.776', 'c': '102.786'}},
  {'complete': True,
   'volume': 52,
   'time': '2021-01-06T02:17:00.000000000Z',
   'mid': {'o': '102.786', 'h': '102.792', 'l': '102.775', 'c': '102.786'}},
  {'complete': True,
   'volume': 36,
   'time': '2021-01-06T02:18:00.000000000Z',
   'mid': {'o': '102.787', 'h': '102.803', 'l': '102.784',

In [9]:

# ストリーミングの最初の1件目のデータを確認
r.response['candles'][0]

{'complete': True,
 'volume': 86,
 'time': '2021-01-06T02:13:00.000000000Z',
 'mid': {'o': '102.782', 'h': '102.794', 'l': '102.780', 'c': '102.790'}}

In [10]:

# 為替レートのdictをDataFrameへ変換
rate = pd.DataFrame.from_dict({r.response['candles'][i]['time']: r.response['candles'][i]['mid']
                           for i in range(0,len(r.response['candles']))
                           for j in r.response['candles'][i]['mid'].keys()},
                       orient='index',
                      )

In [11]:

# インデックスの日付を綺麗にする
rate.index = pd.to_datetime(rate.index)
 
# 念のためDataFrameの確認
rate.head()

,o,h,l,c
2021-01-06 02:13:00+00:00,102.782,102.794,102.780,102.790
2021-01-06 02:14:00+00:00,102.790,102.792,102.780,102.788
2021-01-06 02:15:00+00:00,102.786,102.800,102.782,102.785
2021-01-06 02:16:00+00:00,102.786,102.788,102.776,102.786
2021-01-06 02:17:00+00:00,102.786,102.792,102.775,102.786


In [26]:
#mplfiianceの追加
import mplfinance as mpf
#import DataFrame as df
    
#DataFrameの時刻用の列をDateTimeインデックスに変換
rate.set_index(pd.to_datetime(rate['time']), inplace=True)
          
#要素の属性をfloat型に変換
new_df = rate[['o', 'h', 'l', 'c',  ]].astype(float)
    
#列名を指定された名前に変更
new_df.columns = ['Open', 'High', 'Low', 'Close']
    
#チャートの表示
mpf.plot(new_df,type='candle')

KeyError: 'time'

In [21]:
#df = rate.loc[:,['Open','High','Low','Close','Volume']].sort_values('Date')

#import mplfinance as mpf
#mpf.plot(df, type='candle', style='charles', volume=True)

KeyError: "None of [Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')] are in the [columns]"

In [15]:
#最新のライブラリであるmplfinanceでローソク足を書くべく調査中
# ローソク足チャートをMatplotlibで作る
import matplotlib.pyplot as plt
import mplfinance as mpf
#import matplotlib.finance as mpf
#import mpl_finance as mpf
from matplotlib import ticker
import matplotlib.dates as mdates
#from mplfinance.original_flavor import candlestick_ohlc

In [20]:
#mpf.plot(rate, Open=rate.o.values, Close=rate.c.values, Low=rate.l.values, High=rate.h.values,type='candle')

KeyError: 'Unrecognized kwarg="Open"'

In [16]:

# データフレームからローソク足チャートへ
def candlechart(data, width=0.8):
    fig, ax = plt.subplots()
    # ローソク足
    mpf.candlestick_ohlc(ax, opens=data.o.values, closes=data.c.values,
                          lows=data.l.values, highs=data.h.values,
                          width=width, colorup='r', colordown='b')
 
    # x軸を時間にする
    xdate = data.index
    ax.xaxis.set_major_locator(ticker.MaxNLocator(6))
 
    def mydate(x, pos):
        try:
            return xdate[int(x)]
        except IndexError:
            return ''
 
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(mydate))
    ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
    
 
    fig.autofmt_xdate()
    fig.tight_layout()
 
    return fig, ax

In [17]:

# ローソク足チャートのプロッティング
candlechart(rate)

AttributeError: module 'mplfinance' has no attribute 'candlestick_ohlc'